In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import os
import sys
from pathlib import Path
import geopandas as gpd
import yaml
import plotly.graph_objects as go


In [64]:
# fake small dataframe
df = pd.DataFrame(np.random.randint(0, 10,size=(10, 4)), columns=list('ABCD')).T

print(df)
df.reindex(pd.Index(["D", "C"] + df.index.drop(["C","D"]).tolist()))




   0  1  2  3  4  5  6  7  8  9
A  7  8  5  2  8  7  7  4  2  5
B  0  0  6  9  4  0  2  7  5  5
C  6  3  1  2  0  9  2  9  2  5
D  2  4  1  9  3  8  2  3  5  6


,0,1,2,3,4,5,6,7,8,9
D,2,4,1,9,3,8,2,3,5,6
C,6,3,1,2,0,9,2,9,2,5
A,7,8,5,2,8,7,7,4,2,5
B,0,0,6,9,4,0,2,7,5,5


In [65]:
idx = pd.IndexSlice

file = Path.cwd().parent.parent.parent / "cluster_data" / "elec_data" / "csvs" / "nodal_capacities.csv" 
shapes = Path.cwd().parent.parent.parent / "cluster_data" / "resources" / "regions_onshore_elec_s_72.geojson"
shapes = gpd.read_file(shapes).set_index("name")
shapes["values"] = 1.
shapes.drop("geometry", axis=1, inplace=True)

shapes = shapes.groupby(shapes.index.str[:2]).sum()

with open(Path.cwd().parent / "config" / "config.yaml") as f:
    tech_colors = yaml.safe_load(f)["plotting"]["tech_colors"]

n_countries = 10
n_carriers = 15

df = pd.read_csv(file, index_col=[0,1,2], header=[4,5,6]).fillna(0)

# diff = df.loc[:, idx[["1500"], :, ["static"]]]
diff = df.iloc[:, 1:2]

# for c in diff.index.get_level_values(0).unique():
# if c != "links":
#     continue

ss = diff.loc[idx[["links", "generators"], :, :]]

totals = ss.groupby(level=1).sum()
totals = totals.groupby(totals.index.str[:2]).sum()

totals = totals.loc[totals.index.str[:2].isin(shapes.index)].iloc[:,0]
totals = totals.sort_index()
totals.name = "totals"

ss = ss.unstack()

if isinstance(ss.index, pd.MultiIndex):
    ss.index = ss.index.get_level_values(1)

ss = ss.groupby(ss.index.str[:2]).sum()
ss.columns = ss.columns.get_level_values(3)

# print(ss.shape)
# print(ss.iloc[:5,:5])
# print(totals.head())

ss = (
    ss
    .loc[totals.index]
    .T
    .fillna(0.)
    .drop_duplicates()
)
# print("before")
# print(ss)


country_totals = ss.sum(axis=0).sort_values(ascending=False)
carrier_totals = ss.sum(axis=1).sort_values(ascending=False)

selected_countries = country_totals.iloc[:n_countries].index
selected_carriers = carrier_totals.iloc[:n_carriers].index
# df.reindex(pd.Index(["D", "C"] + df.index.drop(["C","D"]).tolist()))
selected_carriers = pd.Index(["geothermal heat", "geothermal heat elec"] + selected_carriers.drop(["geothermal heat", "geothermal heat elec"]).tolist())

print(selected_carriers)

# ss = ss.loc[selected_countries, selected_carriers]
ss = ss.loc[selected_carriers, selected_countries]
# print("after")
 #print(ss)

# print("resulting shape")
# print(ss.shape)

totals = ss.sum()

ss = ss.div(totals, axis=1).mul(100)

# print("ss post div")
# print(ss)


print(ss.index.to_frame().shape)

totals /= totals.sum()

# ss.index = ss.index.get_level_values(3)
print(ss.columns)

ss = ss.T.sort_values(by="geothermal heat", ascending=False).T
totals = totals.loc[ss.columns]

ss = ss.drop_duplicates()
widths = totals.values.flatten()

fig = go.Figure()

for i, carrier in enumerate(ss.index):
    if carrier == "DC":
        continue

    dff = ss.loc[carrier]

    fig.add_trace(
        go.Bar(
            x=np.cumsum(widths) - widths/2.,
            y=dff.values,
            width=widths,
            marker_color=tech_colors[carrier],
            name=carrier,
        )
    )

fig.update_xaxes(
    tickvals=np.cumsum(widths)-widths/2.,
    # ticktext= ["%s" % l for l in totals.index]
    ticktext= [l for l in totals.index]
)

fig.update_xaxes(range=[0, widths])
fig.update_yaxes(range=[0, 100])

fig.update_layout(barmode='stack', width=1600, height=500, xaxis_title="Country", yaxis_title="Share of Total Generation [%]")

# fig.write_image("mara.png")
fig.show()


Index(['geothermal heat', 'geothermal heat elec', 'solar', 'onwind',
       'H2 Electrolysis', 'urban central solar thermal',
       'electricity distribution grid', 'BEV charger',
       'urban central water tanks charger', 'Fischer-Tropsch', 'solar rooftop',
       'residential urban decentral water tanks charger',
       'urban central water tanks discharger',
       'residential urban decentral water tanks discharger', 'offwind-dc'],
      dtype='object')
(15, 1)
Index(['ES', 'DE', 'GB', 'FR', 'IT', 'PL', 'NL', 'IE', 'BE', 'GR'], dtype='object')


In [24]:
year = ['2019', '2020', '2021', '2022']

data = {'brand 1': [20, 18, 14, 10],
       'brand 2': [10, 15, 20, 22],
       'brand 3': [6,   8, 10, 12]
       }

df = pd.DataFrame.from_dict(data)

df = df.T

df.columns = year

for c in df.columns:
    df[c+'_%'] = df[c].apply(lambda x: (x / df.loc[:,c].sum()) * 100)

widths = np.array([sum(df['2019']), sum(df['2020']), sum(df['2021']), sum(df['2022'])])
marker_colors = {'brand 1': 'darkblue', 'brand 2': 'darkgreen', 'brand 3': 'crimson'}

fig1 = go.Figure()

print(df)
print("===============")

for idx in df.index:
    dff = df.filter(items=[idx], axis=0)

    print(dff)
    print("===============")
    print(dff[dff.columns[4:]].values[0])
    fig1.add_trace(go.Bar(
        x=np.cumsum(widths) - widths,
        y=dff[dff.columns[4:]].values[0],
        width=widths,
        marker_color=marker_colors[idx],
        text=['{:.2f}%'.format(x) for x in dff[dff.columns[4:]].values[0]],
        name=idx
        )
    )

fig1.update_xaxes(
    tickvals=np.cumsum(widths)-widths,
    ticktext= ["%s<br>%d" % (l, w) for l, w in zip(year, widths)]
)
fig1.update_xaxes(range=[0, widths])
fig1.update_yaxes(range=[0, 100])
fig1.update_layout(barmode='stack')
fig1.show()

         2019  2020  2021  2022     2019_%     2020_%     2021_%     2022_%
brand 1    20    18    14    10  55.555556  43.902439  31.818182  22.727273
brand 2    10    15    20    22  27.777778  36.585366  45.454545  50.000000
brand 3     6     8    10    12  16.666667  19.512195  22.727273  27.272727
         2019  2020  2021  2022     2019_%     2020_%     2021_%     2022_%
brand 1    20    18    14    10  55.555556  43.902439  31.818182  22.727273
[55.55555556 43.90243902 31.81818182 22.72727273]
         2019  2020  2021  2022     2019_%     2020_%     2021_%  2022_%
brand 2    10    15    20    22  27.777778  36.585366  45.454545    50.0
[27.77777778 36.58536585 45.45454545 50.        ]
         2019  2020  2021  2022     2019_%     2020_%     2021_%     2022_%
brand 3     6     8    10    12  16.666667  19.512195  22.727273  27.272727
[16.66666667 19.51219512 22.72727273 27.27272727]


In [21]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

year = ['2019', '2020', '2021', '2022']
data = {'brand 1': [20, 18, 14, 10],
       'brand 2': [10, 15, 20, 22],
       'brand 3': [6,   8, 10, 12]
       }
df = pd.DataFrame.from_dict(data)

df = df.T

df.columns = year
for c in df.columns:
    df[c+'_%'] = df[c].apply(lambda x: (x / df.loc[:,c].sum()) * 100)

widths = np.array([sum(df['2019']), sum(df['2020']), sum(df['2021']), sum(df['2022'])])
marker_colors = {'brand 1': 'darkblue', 'brand 2': 'darkgreen', 'brand 3': 'crimson'}

fig1 = go.Figure()

for idx in df.index:
    dff = df.filter(items=[idx], axis=0)
    fig1.add_trace(go.Bar(
        x=np.cumsum(widths) - widths,
        y=dff[dff.columns[4:]].values[0],
        width=widths,
        marker_color=marker_colors[idx],
        text=['{:.2f}%'.format(x) for x in dff[dff.columns[4:]].values[0]],
        name=idx
    )
)

fig1.update_xaxes(
    tickvals=np.cumsum(widths)-widths,
    ticktext= ["%s<br>%d" % (l, w) for l, w in zip(year, widths)]
)

fig1.update_xaxes(range=[0, widths])
fig1.update_yaxes(range=[0, 100])

fig1.update_layout(barmode='stack')

#fig1.write_image('test_1.png')
fig1.show()

0    36
1    41
2    44
3    44
dtype: int64
